This notebook prepare the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

In [1]:
import numpy as np
import os
os.environ['LOG_LEVEL'] = 'INFO'

from astropy.table import Table

from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

In [2]:
SUFFIX = '20180427'
master_catalogue = Table.read("../../dmu32/dmu32_HDF-N/data/HDF-N_{}_cigale.fits".format(SUFFIX))

In [3]:
len(master_catalogue)

130679

# Best sources

Define a good far-IR measurement as:
- an existing flux in the band;
- the flag from XID+ must not be set;
- the signal to noise ratio must be over 2.

In [4]:
good = {}
for band in ['spire_250', 'spire_350', 'spire_500']:
    good[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]) & 
                                   ~master_catalogue['flag_{}'.format(band)])
    good[band][good[band]] &= (master_catalogue[good[band]]['f_{}'.format(band)] /
                              master_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)

We will keep only sources with at leat 2 good far-IR measurements (we may actually use less sources are not all may have a redshift).

In [5]:
combined_good = np.sum(list(good.values()), axis=0) >= 2

In [6]:
print("Number of good sources: {}".format(np.sum(combined_good)))

Number of good sources: 0


In [7]:
# Only sources with at least two optical and at least two near infrared detections
# HDF-N masterlist ignored some of the nir fluxes
#optnir = ((master_catalogue['flag_optnir_det'] == 3) 
#          | (master_catalogue['flag_optnir_det'] == 7))

In [8]:
nb_optical_flux = (
    # PanSTARRS
    1 * ~np.isnan(master_catalogue['f_gpc1_g']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_r']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_i']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_z']) +
    1 * ~np.isnan(master_catalogue['f_gpc1_y']) +
    1 * ~np.isnan(master_catalogue['f_suprime_r']) + 
    1 * ~np.isnan(master_catalogue['f_suprime_ip']) +
    1 * ~np.isnan(master_catalogue['f_suprime_zp'])
)

nb_nir_flux = (
    1 * ~np.isnan(master_catalogue['f_wircam_ks']) +
    1 * ~np.isnan(master_catalogue['f_quirc_hk']) +
    1 * ~np.isnan(master_catalogue['f_moircs_k'])
)

has_optical_flux = nb_optical_flux >= 2
has_nir_flux = nb_nir_flux >= 2

optnir = has_optical_flux & has_nir_flux

print("Number of optnir sources: {}".format(np.sum(optnir)))

Number of optnir sources: 21759


# Main catalogue for CIGALE

In [9]:
#best_catalogue = master_catalogue[combined_good].copy()
best_catalogue = master_catalogue[combined_good].copy()

In [10]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

In [11]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [12]:
# Convertion to CIGALE format
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

## Band selection

We want to use only one filter for similar bands. We define an order of preference and set to NaN the flux in the lower prefered bands when a prefered band is available. Some band may have a 0 flux, we set there values to NaN.

In [13]:

u_bands = ["mosaic_u"]
g_bands = [              "gpc1_g"]
r_bands = ["suprime_r",  "gpc1_r" ]
i_bands = ["suprime_ip", "gpc1_i" ]
z_bands = ["suprime_zp", "gpc1_z"]
y_bands = [              "gpc1_y"]

def remove_unneeded_fluxes(list_of_bands):
    for band_idx, band in enumerate(list_of_bands[:-1]):
        mask = ~np.isnan(best_catalogue[band])
        for lower_band in list_of_bands[band_idx+1:]:
            best_catalogue[lower_band][mask] = np.nan
            best_catalogue["{}_err".format(lower_band)][mask] = np.nan

In [14]:
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)

In [15]:
best_catalogue.write("data_tmp/HDF-N_cigale_best_extcor_{}.fits".format(SUFFIX), overwrite=True)

# Catalogue using spectroscopic redshift

In [16]:
best_catalogue = master_catalogue[combined_good].copy()

In [17]:
best_catalogue.remove_column("redshift")
best_catalogue["zspec"].name = "redshift"

In [18]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

In [19]:
print("Number of sources with z-spec: {}".format(len(best_catalogue)))

Number of sources with z-spec: 2994


In [20]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [21]:
# Convertion to CIGALE format
os.environ['LOG_LEVEL'] = 'INFO'
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 187 sources, the band mosaic_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 94 sources, the band acs_f435w should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 94 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 35 sources, the band gpc1_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 10 sources, the band suprime_v should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 13 sour

In [22]:
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)

In [23]:
best_catalogue.write("data_tmp/HDF-N_cigale_best_extcor_zspec_{}.fits".format(SUFFIX), overwrite=True)